In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn
import torchmetrics
import sklearn
from sklearn.model_selection import *
from sklearn.metrics import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
device = torch.device('cpu')

In [4]:
device

device(type='cpu')

In [5]:
dataset = pd.read_csv('creditcard.csv')

In [6]:
X = dataset.iloc[:,:-1]
Y = dataset.iloc[:,-1]

In [7]:
class_weights=sklearn.utils.class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y),y=Y)
class_weights=torch.tensor(class_weights,dtype=torch.float32)
class_weights

tensor([  0.5009, 289.4380])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,random_state=1,stratify=Y)

In [9]:
X_train_torch = torch.tensor(X_train.values).float()
X_test_torch = torch.tensor(X_test.values).float()
Y_train_torch = torch.tensor(y_train.values).long()
Y_test_torch = torch.tensor(y_test.values).long()

In [10]:
train_data = torch.utils.data.TensorDataset(X_train_torch,Y_train_torch)
test_data = torch.utils.data.TensorDataset(X_test_torch,Y_test_torch)

In [11]:
train_loader = torch.utils.data.DataLoader(train_data,batch_size=16,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=16,shuffle=True)

In [12]:
class DNN(torch.nn.Module):
    def __init__(self,n_inputs):
        super(DNN,self).__init__()
        self.hidden1 = torch.nn.Linear(n_inputs,100)
        torch.nn.init.kaiming_normal_(self.hidden1.weight)
        self.norm1 = torch.nn.BatchNorm1d(100)
        self.hidden2 = torch.nn.Linear(100,150)
        torch.nn.init.kaiming_normal_(self.hidden2.weight)
        self.norm2 = torch.nn.BatchNorm1d(150)
        self.hidden3 = torch.nn.Linear(150,80)
        torch.nn.init.kaiming_normal_(self.hidden3.weight)
        
    def forward(self, X):
        X = self.hidden1(X)
        X = torch.nn.functional.selu(self.norm1(X))
        X = self.hidden2(X)
        X = torch.nn.functional.selu(self.norm2(X))
        X = self.hidden3(X)
        X = torch.nn.functional.logsigmoid(X)
        return X

In [13]:
model = DNN(len(X_train.columns)).to(device)

In [14]:
model

DNN(
  (hidden1): Linear(in_features=30, out_features=100, bias=True)
  (norm1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hidden2): Linear(in_features=100, out_features=150, bias=True)
  (norm2): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (hidden3): Linear(in_features=150, out_features=80, bias=True)
)

In [15]:
def model_summary(model):
    model_params_list = list(model.named_parameters())
    print("----------------------------------------------------------------")
    line_new = "{:>20}  {:>25} {:>15}".format("Layer.Parameter", "Param Tensor Shape", "Param #")
    print(line_new)
    print("----------------------------------------------------------------")
    for elem in model_params_list:
        p_name = elem[0] 
        p_shape = list(elem[1].size())
        p_count = torch.tensor(elem[1].size()).prod().item()
        line_new = "{:>20}  {:>25} {:>15}".format(p_name, str(p_shape), str(p_count))
        print(line_new)
    print("----------------------------------------------------------------")
    total_params = sum([param.nelement() for param in model.parameters()])
    print("Total params:", total_params)
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params:", num_trainable_params)
    print("Non-trainable params:", total_params - num_trainable_params)
    return

In [16]:
model_summary(model)

----------------------------------------------------------------
     Layer.Parameter         Param Tensor Shape         Param #
----------------------------------------------------------------
      hidden1.weight                  [100, 30]            3000
        hidden1.bias                      [100]             100
        norm1.weight                      [100]             100
          norm1.bias                      [100]             100
      hidden2.weight                 [150, 100]           15000
        hidden2.bias                      [150]             150
        norm2.weight                      [150]             150
          norm2.bias                      [150]             150
      hidden3.weight                  [80, 150]           12000
        hidden3.bias                       [80]              80
----------------------------------------------------------------
Total params: 30830
Trainable params: 30830
Non-trainable params: 0


In [17]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.003)

In [18]:
print('--------------------------------------------------------------------------------------------------------------')

for epoch in range(1,10):
    
    model.train()
    running_loss = 0
    for i, (inputs_train,targets_train) in enumerate(train_loader):
        optimizer.zero_grad()
        yhat_train = model.forward(inputs_train)
        loss_train = criterion(yhat_train,targets_train)
        loss_train.backward()
        optimizer.step()
        running_loss += loss_train
    
    model.eval()
    yhat_test = model(X_test_torch)
    actuals = Y_test_torch
    loss_test = criterion(yhat_test,actuals)
    _, yhat_test = torch.max(yhat_test,1)
                
    accuracy = torchmetrics.functional.accuracy(actuals,yhat_test)
    precision = torchmetrics.functional.precision(actuals,yhat_test)
    recall = torchmetrics.functional.recall(actuals,yhat_test)
    print('Epoch: {}, Loss train: {:.4f}, Loss test: {:.4f}, Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}'.format(
            epoch,running_loss/len(train_loader),loss_test,accuracy,precision,recall))
    print('--------------------------------------------------------------------------------------------------------------')

--------------------------------------------------------------------------------------------------------------
Epoch: 1, Loss train: 0.0330, Loss test: 0.0060, Accuracy: 0.9986, Precision: 0.9986, Recall: 0.9986
--------------------------------------------------------------------------------------------------------------
Epoch: 2, Loss train: 0.0060, Loss test: 0.0231, Accuracy: 0.9992, Precision: 0.9992, Recall: 0.9992
--------------------------------------------------------------------------------------------------------------
Epoch: 3, Loss train: 0.0054, Loss test: 0.0036, Accuracy: 0.9993, Precision: 0.9993, Recall: 0.9993
--------------------------------------------------------------------------------------------------------------
Epoch: 4, Loss train: 0.0050, Loss test: 0.0036, Accuracy: 0.9987, Precision: 0.9987, Recall: 0.9987
--------------------------------------------------------------------------------------------------------------
Epoch: 5, Loss train: 0.0049, Loss test: 

In [19]:
print(confusion_matrix(actuals,yhat_test))

[[56855     9]
 [   21    77]]


In [20]:
print(classification_report(actuals,yhat_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.90      0.79      0.84        98

    accuracy                           1.00     56962
   macro avg       0.95      0.89      0.92     56962
weighted avg       1.00      1.00      1.00     56962

